# Workshop 3: Harmonic solutions

Workshop 2 showed that the heat/diffusion equation could be used on general functions (i.e. heat or positions) to smooth them. As the solution evolved in time, it would get smoother and smoother. In fact, one might wonder what happens at the "end" of such a diffusion, i.e. the steady state? In this setting, the time derivative of the function would vanish, i.e. $\frac{\partial u}{\partial t} = 0$, since $u$ does not change in the steady state. This leads to the [harmonic (or Laplace) equation](https://en.wikipedia.org/wiki/Laplace%27s_equation) $\Delta u = 0$. Functions which satisfy this equation are said to be harmonic. In general, harmonic functions are quite smooth, and we can thus often "enforce" smoothness on desired solutions by making them harmonic. This smoothness is even more obvious when we consider how harmonic functions are minimizers of the [Dirichlet (or smoothness) energy](https://en.wikipedia.org/wiki/Dirichlet_energy)

$$
\Delta u(X) = 0 \longrightarrow u = \min_u \frac{1}{2} \int_{\Omega} || \nabla u(X) ||_2^2 \partial \Omega .
$$

## FEM discretization

1. Discretize the Dirichlet energy using FEM. 
2. Discretize the Laplace equation using FEM.


For the Dirichlet energy, the final result should be $\min_u -\frac{1}{2} u^T L u$, where $L$ is the symmetric part of the Laplacian energy. For the Laplace equation, we simply get $Lu = 0$.

## Solving the Laplace equation

In workshop 1, we learned that the Laplace equation is in fact rank deficient (by 1). We will thus need to impose some Dirichlet boundary conditions on the solution to obtain a unique harmonic function. 

1. Show that the Laplacian is rank deficient (on paper).
2. Confirm that it is numerically rank deficient (even if workshop 1 already asks this).

## Interpolation problems

The "smoothness" minimization framework for thinking about the Laplace equation naturally allows us to formulate interpolation as finding the smoothest scalar function which interpolates the Dirichlet boundary conditions. In other words, 

$$
\min_u -u^T L u \quad\text{s.t.}\quad u_D = d_D 
$$

for some set of Dirichlet nodes $D$ and their corresponding imposed values $d_D$.
Knowing that $-L$ is positive semi-definite with a single missing rank, we need at least $1$ boundary condition, and we can solve this minimization by setting the gradient to $0$, yielding the reduced linear system

$$
L_{uu} u_u = L_{uD} d_D ,
$$

where the subscripts $u,D$ are to be taken as indices to unknown degrees of freedom, and indices to known degrees of freedom (i.e. Dirichlet boundary conditions), respectively.

1. Constrain 2 parts of the mesh to have values $0$ in the first part, and $1$ in the second part. Solve this Dirichlet constrained minimization. We are essentially interpolating between $0$ and $1$ on some complex and curved geometry (the mesh). Much cooler than interpolation on a line segment or in a box!
2. Consider the vector-valued function of displacements from FEM nodal positions as a minimizer to the Dirichlet energy. In other words, we wish to find some smooth displacement field that takes FEM node positions to some smoother positional configuration. Constrain some region of the FEM mesh to be fixed. Constrain another region of the FEM mesh to "move" someplace else, by setting the corresponding Dirichlet conditions in $X,Y,Z$ dimensions to have a non-trivial displacement $[d_X, d_Y, d_Z]$. Solve for the $3$ resulting interpolants and visualize the result. You have essentially implemented [harmonic deformation](https://dl.acm.org/doi/10.1145/1186562.1015772), a popular framework for shape deformation!

## k-Harmonic interpolation

It should be natural to wonder if smoother functions can be obtained in this same framework. In other words, why not minimize change in first derivatives *and* second derivatives *and* higher order derivatives? This leads to the k-harmonic equation

$$
\Delta^k u(X) = 0 ,
$$

where $\Delta^k$ applies the Laplacian $k$ times. Unfortunately, on first impression, it might seem like this would require our discretized $u(X)$ to be $2k$ times differentiable (ignoring that constant functions are differentiable). In other words, our basis functions $\phi_i(X)$ should be $2k$ times differentiable. However, this problem can be solved using only linear basis functions by thinking recursively. We can create auxiliary variables $u^{k-1}(X) = \Delta^{k-1} u(X)$ and start by solving for $\Delta u^{k-1}(X) = 0$. Then, recursively solve $\Delta u^{k-i}(X) = u^{k-i+1}(X)$. Unrolling this loop yields 

$$
\Delta (\dots (\Delta u(X))) = 0 ,
$$

where $u(X) = u^{1}(X)$. Discretizing each intermediate (sub)problem using FEM, we get that 

$$
L u^{k-i} = M u^{k-i+1} ,
$$

as we have shown [here](./README.md), where $u^{k-i+1}(X)$ is considered as a forcing function in the $i^\text{th}$ subproblem. Because we know that FEM functions are interpolating, this must mean that the coefficients in $u^{k-i+1}$ are the actual values of the Laplacian of $u^{k-i}(X)$ at the FEM nodes, and we can obtain such Laplacians via 

$$
M^{-1} L u^{k-i} = u^{k-i+1} .
$$

Thinking recursively again, we realize that $u^{k-i+1}$ was also obtained by applying the matrix $M^{-1} L$ to $u^{k-i+2}$. At the end of this recursion, we get $L u^k = 0$ without any mass matrix term. Unrolling this whole recursion, we end up with the system 

$$
\left[ L M^{-1} \dots L M^{-1} L \dots M^{-1} L \right] u = 0 ,
$$

subject to Dirichlet boundary conditions for a well-defined solution to exist. This system is valid for linear shape functions, even if its solution is of higher order. A more in depth mixed finite elements derivation of this technique, including various other types of boundary conditions is described in [Mixed Finite Elements for Variational Surface Modeling](https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1467-8659.2010.01765.x).

1. What are the properties of the lead matrix $\left[ L M^{-1} \dots L M^{-1} L \dots M^{-1} L \right]$ for $k=1,2,3$?
2. Solve the same interpolation problems as in the previous cell, but now minimizing a k-harmonic energy for $k=1,2,3$. What happens as $k$ increases?
3. What "energy" are we minimizing by solving the $k$-harmonic equation? Try to derive this on paper.

A better smoothness energy is described in [Natural Boundary Conditions for Smoothing in Geometry Processing](https://dl.acm.org/doi/abs/10.1145/3186564) and [A Smoothness Energy without Boundary Distortion for Curved Surfaces](https://dl.acm.org/doi/pdf/10.1145/3377406), among other potential approaches.